In [33]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage,HumanMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time

In [34]:
load_dotenv(find_dotenv("../.env"))

True

In [35]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
# llm=ChatOpenAI(model="gpt-3.5-turbo")

In [36]:
prompt=ChatPromptTemplate.from_template(
    template="Tell me a Short joke about {topic}"
)

In [37]:
outParser=StrOutputParser()

In [38]:
chain=prompt|llm|StrOutputParser()

In [39]:
question={"topic":"Virat Kohli"}

In [40]:
chain.invoke(input=question)

'Why was Virat Kohli bad at hide-and-seek?\n\nBecause every time he hid, the camera found him!'

<h3>Prompt</h3>

In [41]:
promptValue=prompt.invoke(input=question)
promptValue

ChatPromptValue(messages=[HumanMessage(content='Tell me a Short joke about Virat Kohli', additional_kwargs={}, response_metadata={})])

In [42]:
promptValue.to_messages()

[HumanMessage(content='Tell me a Short joke about Virat Kohli', additional_kwargs={}, response_metadata={})]

In [43]:
promptValue.to_string()

'Human: Tell me a Short joke about Virat Kohli'

In [44]:
promptValue.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'prompts', 'chat', 'ChatPromptValue'],
 'kwargs': {'messages': [HumanMessage(content='Tell me a Short joke about Virat Kohli', additional_kwargs={}, response_metadata={})]}}

<h3>Model</h3>

In [45]:
message=llm.invoke(input=promptValue)
message

AIMessage(content='Why did Virat Kohli bring a ladder to the cricket match?\n\nBecause he wanted to reach the next level of his batting!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-d3df0dfa-de04-49c2-b775-99a52294317d-0', usage_metadata={'input_tokens': 8, 'output_tokens': 26, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}})

In [46]:
message=llm.invoke(input=promptValue.to_messages())
message

AIMessage(content='Why did Virat Kohli bring a ladder to the cricket match?\n\nBecause he heard the score needed to go higher!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-04a07236-3e5b-41e1-86f8-b6fad0b85b2c-0', usage_metadata={'input_tokens': 8, 'output_tokens': 24, 'total_tokens': 32, 'input_token_details': {'cache_read': 0}})

In [50]:
message=llm.invoke(input=promptValue.to_string())
message

AIMessage(content='Why did Virat Kohli bring a ladder to the cricket match?\n\nBecause he wanted to raise the bar!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-0e013257-d923-48b4-ab69-f2c658d1ec18-0', usage_metadata={'input_tokens': 10, 'output_tokens': 22, 'total_tokens': 32, 'input_token_details': {'cache_read': 0}})

<h3> Output Parser </h3>

In [51]:
outParser.invoke(input=message)

'Why did Virat Kohli bring a ladder to the cricket match?\n\nBecause he wanted to raise the bar!'

<h3>RAG Example</h3>

In [52]:
embeddings=SentenceTransformerEmbeddings() 

C:\Users\MSI\AppData\Local\Temp\ipykernel_20432\3013256437.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=SentenceTransformerEmbeddings()
C:\Users\MSI\AppData\Local\Temp\ipykernel_20432\3013256437.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=SentenceTransformerEmbeddings()
c:\Users\MSI\anaconda3\envs\lcnenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readth

In [54]:
vectorStore=Chroma.from_texts(
    texts=["Harrison worked at kensho","bears like to eat honey"],
    embedding=embeddings
)

In [55]:
vectorStore.as_retriever()

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000170998EF410>, search_kwargs={})

In [56]:
template="""
    Answer the question based only on the following context:
    {context}
    Question: {question}
"""

In [57]:
prompt=ChatPromptTemplate.from_template(
    template=template)

In [65]:
vectorStore.as_retriever().invoke(input="Who likes honey?")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='Harrison worked at kensho')]

In [71]:
RunnablePassthrough().invoke({"context":"How are you doing?"})

{'context': 'How are you doing?'}

In [61]:
setupAndRetrieval=RunnableParallel(
    {
        "context":vectorStore.as_retriever(),
        "question":RunnablePassthrough()
    }
)

In [62]:
chain=setupAndRetrieval|prompt|llm|outParser

In [63]:
question="Who worked at Kensho?"

In [64]:
k=time.time()
chain.invoke(input=question)
print(f"Time Taken: {time.time()-k} seconds")

Time Taken: 0.7396340370178223 seconds


<h3>Without Using Runnable Parallel </h3>

In [66]:
chainTest=RunnablePassthrough.assign(context=itemgetter("question")|vectorStore.as_retriever())|prompt|llm|outParser

In [67]:
k=time.time()
chainTest.invoke(input={"question":question})
print(f"Time Taken: {time.time()-k} seconds")

Time Taken: 0.8226275444030762 seconds


<h3> Retriever Part </h3>

In [68]:
vectorStore.as_retriever().invoke(input=question)

[Document(metadata={}, page_content='Harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [69]:
retrievedContent=setupAndRetrieval.invoke(input=question)
retrievedContent

{'context': [Document(metadata={}, page_content='Harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'Who worked at Kensho?'}

<h3>Prompt Part</h3>

In [72]:
promptValue=prompt.invoke(input=retrievedContent)
promptValue

ChatPromptValue(messages=[HumanMessage(content="\n    Answer the question based only on the following context:\n    [Document(metadata={}, page_content='Harrison worked at kensho'), Document(metadata={}, page_content='bears like to eat honey')]\n    Question: Who worked at Kensho?\n", additional_kwargs={}, response_metadata={})])

In [73]:
promptValue.to_messages()

[HumanMessage(content="\n    Answer the question based only on the following context:\n    [Document(metadata={}, page_content='Harrison worked at kensho'), Document(metadata={}, page_content='bears like to eat honey')]\n    Question: Who worked at Kensho?\n", additional_kwargs={}, response_metadata={})]

In [74]:
print(promptValue.to_string())

Human: 
    Answer the question based only on the following context:
    [Document(metadata={}, page_content='Harrison worked at kensho'), Document(metadata={}, page_content='bears like to eat honey')]
    Question: Who worked at Kensho?



<h3> LLM </h3>

In [75]:
message=llm.invoke(input=promptValue)
message

AIMessage(content='Harrison worked at kensho.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-0c5bb4c3-97e1-4682-9152-9c4dfb546dd9-0', usage_metadata={'input_tokens': 55, 'output_tokens': 8, 'total_tokens': 63, 'input_token_details': {'cache_read': 0}})

<h3>Output Parser</h3>

In [76]:
output=outParser.invoke(input=message)
output

'Harrison worked at kensho.'